<a href="https://colab.research.google.com/github/ramkumar27072006/AMR_2nd/blob/main/Cnn_Gene_Sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files

# Upload the file
uploaded = files.upload()


Saving gene sequence(AMR dataset).csv to gene sequence(AMR dataset).csv


In [4]:
import pandas as pd

# Assuming the file name is 'gene sequence(AMR dataset).csv', use the exact file name
df = pd.read_csv('gene sequence(AMR dataset).csv')

# Show the first few rows of the dataframe
print(df.head())


            Sequence ID                                        Description  \
0        JNOG01000183.1  Escherichia coli strain CS03 Contig_352, whole...   
1  NZ_JACCNT010000135.1  Escherichia coli strain PE01_343 .PE01_343.135...   
2     NZ_JJLU01000172.1  Escherichia coli 1-250-04_S3_C1 e125004S3C1.co...   
3     NZ_JNPX01000205.1  Escherichia coli 2-210-07_S3_C2 e221007S3C2.co...   
4     NZ_JNQV01000096.1  Escherichia coli 2-427-07_S1_C1 e242707S1C1.co...   

                                        DNA Sequence  Class  
0  TTTTTTCCTGTTGTCATCTACACTTAACAAAAATACAGCAAGGAAA...      0  
1  CTGCTCGATCTGTTGCCACCAGTCGCGAATGTCGTCCAGCGGCTGC...      0  
2  CCCCCGGGGGGGGGGGGGGGGGGGGGGGGGGGAAAACGACCTCTCA...      0  
3  TGGGGGGGGGGGGGGGGGGGGGGGGGGGGAAAACGACCTCTCAGGA...      0  
4  CGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAAAAACGACCTCT...      0  


In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer

# Function to extract k-mers (k=4)
def get_kmers(sequence, k=4):
    return ["".join(sequence[i:i+k]) for i in range(len(sequence) - k + 1)]

# Convert sequences into k-mer lists
df["kmers"] = df["Sequence ID"].apply(lambda x: " ".join(get_kmers(x)))


# Apply TF-IDF vectorization
vectorizer = TfidfVectorizer(analyzer=lambda x: x.split(), ngram_range=(4,4))
X_tfidf = vectorizer.fit_transform(df["kmers"])

# Labels
y = df["Class"].values

print("TF-IDF Feature Matrix Shape:", X_tfidf.shape)





import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Define CNN Model
model_cnn = Sequential([
    Conv1D(filters=128, kernel_size=5, activation="relu", input_shape=(X_tfidf.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")  # Binary classification
])

model_cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Reshape TF-IDF feature matrix for CNN
X_cnn = X_tfidf.toarray().reshape(X_tfidf.shape[0], X_tfidf.shape[1], 1)

# Train the CNN Model
model_cnn.fit(X_cnn, y, epochs=10, batch_size=32, validation_split=0.2)


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:533: UserWarning: The parameter 'ngram_range' will not be used since 'analyzer' is callable'
  warnings.warn(


TF-IDF Feature Matrix Shape: (607, 2419)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 403ms/step - accuracy: 0.7129 - loss: 0.5567 - val_accuracy: 0.7541 - val_loss: 0.6271
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 435ms/step - accuracy: 0.8351 - loss: 0.3202 - val_accuracy: 0.7377 - val_loss: 0.6261
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 338ms/step - accuracy: 0.9543 - loss: 0.1664 - val_accuracy: 0.7541 - val_loss: 0.8650
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 400ms/step - accuracy: 0.9837 - loss: 0.0700 - val_accuracy: 0.7131 - val_loss: 0.8938
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 350ms/step - accuracy: 1.0000 - loss: 0.0293 - val_accuracy: 0.7213 - val_loss: 1.0569
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 332ms/step - accuracy: 1.0000 - loss: 0.0144 - val_accuracy: 0.6721 - val_loss: 1.0742
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 12s 447ms/step - accuracy: 1.0000 - loss: 0.0092 - val_accuracy: 0.7131 - val_loss: 1.2380
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 310ms/step - accuracy: 1.0000 - loss: 0.0054 - val_accuracy: 

In [6]:
new_sequence_kmers = " ".join(get_kmers("ATGCGTACGTAGCTAGC"))
new_sequence_tfidf = vectorizer.transform([new_sequence_kmers]).toarray().reshape(1, -1, 1)
cnn_prediction = model_cnn.predict(new_sequence_tfidf)

print("CNN Prediction:", cnn_prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
CNN Prediction: [[0.95005983]]


In [8]:
import numpy as np

def predict_amr_cnn(sequence, model, vectorizer, k=4):
    """Predicts AMR class using trained CNN model"""
    # Convert sequence to k-mers
    sequence_kmers = " ".join(get_kmers(sequence, k))

    # Convert k-mers into TF-IDF feature vector
    sequence_tfidf = vectorizer.transform([sequence_kmers]).toarray().reshape(1, -1, 1)

    # Make prediction
    prediction = model.predict(sequence_tfidf)

    # Convert probability to class label (threshold = 0.5)
    predicted_class = 1 if prediction[0][0] >= 0.5 else 0

    return predicted_class, prediction[0][0]  # Return class and probability


In [10]:
# Example new gene sequence
new_sequence = "GGGGGCCGCCTTCGCCACCGGTATTCCTCCAGATCTCTACGCATTTCACCGCTACACCTGGAATTCTACCCCCCTCTACGAGACTCAAGCTTGCCAGTATCAGATGCAGTTCCCAGGTTGAGCCCGGGGATTTCACATCTGACTTAACAAACCGCCTGCGTGCGCTTTACGCCCAGTAATTCCGATTAACGCTTGCACCCTCCGTATTACCGCGGCTGCTGGCACGGAGTTAGCCGGTGCTTCTTCTGCGGGTAACGTCAATGAGCAAAGGTATTAACTTTACTCCCTTCCTCCCCGCTGAAAGTACTTTACAACCCGAAGGCCTTCTTCATACACGCGGCATGGCTGCATCAGGCTTGCGCCCATTGTGCAATATTCCCCACTGCTGCCTCCCGTAGGAGTCTGGACCGTGTCTCAGTTCCAGTGTGGCTGGTCATCCTCTCAGACCAGCTAGGGATCGTCGCCTAGGTGAGCCGTTACCCCACCTACTAGCTAATCCCATCTGGGCACATCCGATGGCAAGAGGCCCGAAGGTCCCCCTCTTTGGTCTTGCGACGTTATGCGGTATTAGCTACCGTTTCCAGTAGTTATCCCCCTCCATCAGGCAGTTTCCCAGACATTACTCACCCGTCCGCCACTCGTCAGCGAANCAGCAAGCTGGTTCCTGTTACCGTTCGACTGCATGTGTTANGCCTGCCGC"  # Replace with any sequence

# Make prediction
predicted_class, confidence = predict_amr_cnn(new_sequence, model_cnn, vectorizer)

# Print results
print(f"Predicted Class: {'AMR (1)' if predicted_class == 1 else 'Non-AMR (0)'}")
print(f"Prediction Confidence: {confidence:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Predicted Class: AMR (1)
Prediction Confidence: 0.9501
